In [1]:
import numpy as np
from time import time
from sklearn.metrics.pairwise import cosine_similarity
from cuml.metrics import pairwise_distances
from scipy import sparse
import cupy as cp

## 协同过滤
协同过滤(Collaborative Filtering)是一种经典的且被工业界广泛使用推荐算法。
狭义的协同过滤定义为：通过收集大量其他**相似**用户的偏好或品味信息（**协同**）来自动地对预测（**过滤**）当前用户的兴趣。
广义的协同过滤则定义为：使用多个主体，观点，数据源进行合作来过滤信息或者模式的过程。

协同过滤算法分为两类：

* 基于用户的协同过滤：构建一个 user-item 矩阵，然后寻找该用户的相似用户在 user-item 矩阵中的信息来预测该用户的偏好结果。
* 基于目标项目（item）的协同过滤：构建一个 item-item 矩阵来衡量不同项目之间的关联度，然后使用该矩阵来推导用户的偏好。

这里主要是实现一个基于用户的协同过滤算法，基于用户的协同过滤算法的一般定义为（用户 $i$ 对项目 $m$ 的打分）：

$$score(i, m) = \text{agg}_{j \in N}(score(j, m))$$
其中 $N$ 表示 $i$ 的某个邻域（一般以相似度作为度量），$agg$ 用户对该邻域内所有打分结果进行聚合运算（i.e., 加权平均），包括一些预处理（归一化，收缩，等）。

本节将要实现的算法的核心公式为（以预测用户 $i$ 对电影 $m$ 的偏好度为例）：

$$score(i, m) = \frac{\sum_{k \in N(i, m)} sim(X(i), X(k)) \times score(k, m)}{\sum_{k \in N(i, m)} sim(X(i), X(k))}$$
其中 $X(i)$ 代表用户 $i$ 对所有电影的打分（当然要预测的电影的打分是为 0 的）向量，$N(i, m)$ 在本例中简单选用为所有给电影 $m$ 打了分的用户，相似度量采用**余弦距离（cosine distance）**：

$$sim(X(i), X(k)) = \frac{\langle X(i), X(k) \rangle}{\lVert X(i) \rVert \lVert X(k) \rVert}$$

In [2]:
def read_data(filename):
    user2id = {}
    st = time()
    X = np.zeros((10000, 10000))
    with open(filename) as f:
        for l in f.readlines():
            # The users of X_train and X_test are same
            # user: unique ineter ids with #user = 10000, but the id not in [0, 10000] and may appear randomly
            # movie: unique integer ids in range [1, 10000]
            # rate: unique integer rate in range [1, 5]
            # 训练集和测试集中的数据并不是规整的按电影来划分的，也就是说对于某个电影，训练集和测试集都有一部分用户打分，
            # 不过这些用户不重叠，我们的任务便是用训练集中对该电影的打分以及对应用户的信息来预测测试集中对该电影的打分
            # 因为数据规模不是很大，矩阵虽然是 $10000 \times 10000$ 大小的，不过比较稀疏（训练集稀疏度 6.89%，测试集 1.72%），
            # 所以我们采用全量数据（i.e., 10000 维），并且在后面的计算中尽可能使用稀疏矩阵的形式参与计算。
            user, movie, rate, _ = l.split()
            movie = int(movie) - 1
            rate = int(rate)
            if user not in user2id:
                user2id[user] = len(user2id)
            user = user2id[user]
            X[user, movie] = rate
    print(f'Done in {time() - st}s.')
    return X

X_train = read_data('netflix_train.txt')
X_test = read_data('netflix_test.txt')

Done in 6.223333358764648s.
Done in 1.8312866687774658s.


In [3]:
def rmse_sp(X, X_hat):
    # only nonzero entries in X_test will be used as targets when calculating RMSE
    # 因为我们只需要考虑测试集中非零项，所以我们不能直接使用 sklearn 提供的 RMSE。因为这样会导致
    # 不需要测试的数据也被纳入误差的度量计算中，并且 $n = 10000^2$ 而不是实际的 $n_\text{sample} \approx 1.72M$，
    # 这样可能会导致 RMSE 比实际值偏小。
    st = time()
    test_point = X.astype(np.bool)
    loss = np.sqrt(np.power(
        X_hat[test_point] - X[test_point], 2).mean())
    print(f'rmse_sp took {time() - st:.2f}s.')
    return loss

def rmse_cuda(X, X_hat):
    st = time()
    test_point = X.astype(np.bool)
    loss = cp.sqrt(cp.power(
        X_hat[test_point] - X[test_point], 2).mean()).get().item()
    print(f'rmse_sp took {time() - st:.2f}s.')
    return loss

In [4]:
def pairwise_cos_distance(X):
    X = X.T / X.sum(axis=1)
    return X.T @ X

def coll_filter(train, test, contain_zero_rates=False):
    X_train_csr = sparse.csr_matrix(train)
    X_test_csr = sparse.csr_matrix(test)
    st = time()
    # 首先计算好 $sim(X(i), X(k))$ 避免重复计算
    sim = cosine_similarity(X_train_csr)
    # 计算分子部分（对用户打分结果按照相似度进行加权）
    score = sim @ X_train_csr
    # 计算分母部分（i.e., 归一化因子）
    # nonzero => 1, renormalized, i.e., k \in users who have rated movie j
    # 如果不这样做的话，会有很多没有给 movie j 打分（rate=0）的用户消耗掉一部分的权重（概率分布），
    # 进而造成预测结果偏小
    if contain_zero_rates:
        Z = sim @ np.ones((10000, 1))
    else:
#         X_train_csr[X_train_csr.nonzero()] = 1
        X_train_csr = X_train_csr.astype(np.bool)
        Z = sim @ X_train_csr
    # 因为分母会有一些结果为 0（当所有用户偏好都与该用户差别过大时，或者该用户打分信息很极少时），
    # 所以我们需要对 nan 转化为 0
    score_pred = np.nan_to_num(score / Z)
    print(f'Calculating score_pred done in {time() - st:.2f}s.')
    loss = rmse_sp(X_test, score_pred)
    return loss

def coll_filter_cuda(train, test, contain_zero_rates=False):
    cp.cuda.Device(3).use()
    X_train_cuda = cp.array(train)
    X_test_cuda = cp.array(test)
    st = time()
    sim = pairwise_cos_distance(X_train_cuda)
    score = sim @ X_train_cuda
    if contain_zero_rates:
        Z = sim @ cp.ones((10000, 1))
    else:
        Z = sim @ X_train_cuda.astype(np.bool)
    score_pred = score / Z
    print(f'Calculating score_pred done in {time() - st:.2f}s.')
    loss = rmse_cuda(cp.array(test), score_pred)
    return loss

In [5]:
rmse = coll_filter_cuda(X_train, X_test)
print(f'RMSE: {rmse:.5f}')

rmse_ill_Z = coll_filter_cuda(X_train, X_test, contain_zero_rates=True)
print("RMSE when k includes all users who're not " +
      f"rate this movie: {rmse_ill_Z:.5f}")

Calculating score_pred done in 12.54s.
rmse_sp took 0.09s.
RMSE: 1.02821
Calculating score_pred done in 7.98s.
rmse_sp took 0.09s.
RMSE when k includes all users who're not rate this movie: 2.60669


## 基于梯度下降的矩阵分解算法

因为给定特征矩阵是一个稀疏矩阵，基于低秩假设，特征维度为 10000 的矩阵的特征值也很稀疏（或者很小），于是我们可以将矩阵分解为两个特征维度很小的隐空间特征表达：

$$X_{m \times n} \approx U_{m \times k} V_{n \times k}^T$$
其中 $k \ll n$。

特别的，我们在电影推荐任务中，使用 X_train 构建出矩阵分解后，使用它们的乘积来预测 X_test 中的那部分结果。

这里我们采用梯度下降算法来求解上述分解，目标函数为：

$$J = \frac{1}{2} \lVert A \odot (X - UV^T \rVert_F^2 + \lambda \lVert U \rVert_F^2 + \lambda \lVert V \rVert_F^2)$$
其中 $\odot$ 表示 Hadamard 乘积（逐位乘），$A$ 为指示矩阵，$A_{ij} = 1$ 表示 $X_{ij} 的值已知，$$\lVert A \rVert_F^2 = \sqrt{\sum_i \sum_j A_{ij}}$ 表示矩阵 Frobenius 范数，$\lambda$ 为正则化系数，用于防止过拟合。

对目标函数求偏导，得到：

$$
\frac{\partial J}{\partial U} = (A \odot (UV^T - X))V + 2\lambda U\\
\frac{\partial J}{\partial V} = (A \odot (UV^T - X))^TU + 2\lambda V
$$

所以整个算法的流程为：

1. 初始化 $U, V$ 为较小的随机值
2. 不断循环 $U = U - \alpha \frac{\partial J}{\partial U}, V = V - \alpha \frac{\partial J}{\partial V}$，直到收敛

上述过程中 $\alpha$ 为学习率，一般取值为 [0.0001, 0.1] 之间。

In [6]:
cp.cuda.Device(3).use()

def init_matrix(X, k, init='random', n_features=10000, n_samples=10000,
                seed=1234):
    if init == 'random':
        # 初始化也很重要
        avg = np.sqrt(X.mean() / k)
#         print(f'avg={avg}')
        rng = np.random.RandomState(seed)
        V = avg * rng.randn(k, n_features).astype(X.dtype, copy=False)
        U = avg * rng.randn(n_samples, k).astype(X.dtype, copy=False)
        np.abs(U, out=U)
        np.abs(V, out=V)
        return cp.array(U), cp.array(V)
    elif init == 'nndsvd':
        # https://github.com/scikit-learn/scikit-learn/blob/0fb307bf3/sklearn/decomposition/_nmf.py#L1082
        pass
    else:
        raise Exception('unexcepted init method')

def frobenius_norm_sq(X):
    norm = cp.sum(cp.power(X, 2))
    return norm
    
def objective(diff, U, V, lamb):
    loss = frobenius_norm_sq(diff)
    loss = 0.5 * loss + (frobenius_norm_sq(U) + frobenius_norm_sq(V)) * lamb
    return loss.item()

def fit(X, k, lamb, alpha=1e-3, tol=1e-4, max_iter=500):
    st_fit = time()
    U, V = init_matrix(X, k)
    A = X.copy()
    A = A.astype(np.bool)
    A = cp.array(A)
    X = cp.array(X)
    prev_error = 0.
    for n_iter in range(1, max_iter + 1):
#         print(f'norm(U)={frobenius_norm_sq(U)}, norm(V)={frobenius_norm_sq(V)}')
        diff = A * (U @ V - X)
        # 因为我这里实现的 V 是老师给的公式中的 V.T 所以会有一点不一样。
        update_U = diff @ V.T + 2 * lamb * U
        update_V = U.T @ diff + 2 * lamb * V
        error = objective(-diff, U, V, lamb)
        if n_iter % 10 == 0:
            print(f'Iter: {n_iter}, loss={error:.4f}, ' +
                  f'{(time() - st_fit) / n_iter:.1f}s/epoch.')
        if prev_error != 0 and (prev_error - error) / prev_error <= tol:
            break
        prev_error = error
        U = U - alpha * update_U
        V = V - alpha * update_V
    X_new = U @ V
    return X_new

st = time()
# 学习率太高很容易导致梯度爆炸
X_hat = fit(X_train, k=20, lamb=1, alpha=1e-4, max_iter=200)
print(f'Done in {time() - st:.2f}s.')
rmse = rmse_cuda(cp.array(X_test), X_hat)
print(f'RMSE: {rmse:.5f}.')

Iter: 10, loss=5150756.7619, 0.3s/epoch.
Iter: 20, loss=3677138.7559, 0.2s/epoch.
Iter: 30, loss=3328562.5377, 0.2s/epoch.
Iter: 40, loss=3176529.8730, 0.2s/epoch.
Iter: 50, loss=3093117.5543, 0.2s/epoch.
Iter: 60, loss=3040779.8143, 0.2s/epoch.
Iter: 70, loss=3004176.7839, 0.2s/epoch.
Iter: 80, loss=2975214.4852, 0.2s/epoch.
Iter: 90, loss=2948113.7022, 0.2s/epoch.
Iter: 100, loss=2917469.2723, 0.2s/epoch.
Iter: 110, loss=2880248.3322, 0.2s/epoch.
Done in 23.47s.
rmse_sp took 0.09s.
RMSE: 1.12383.


In [7]:
!nvidia-smi

Sun Dec 13 23:22:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.04    Driver Version: 455.23.04    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:18:00.0 Off |                  N/A |
| 22%   30C    P8    17W / 250W |      3MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:3B:00.0 Off |                  N/A |
| 22%   